# Similitud de documentos (I)

## Cargar dataset generado

In [ ]:
import json
import zlib

import numpy as np
import seaborn as sns

%matplotlib inline

import config

# Import
with open(config.DATASET_MUCHOCINE, 'r') as fd:
    documents = json.loads(zlib.decompress(fd.read()))

## Modelo vectorial

In [ ]:
from collections import Counter

from sklearn.feature_extraction import DictVectorizer

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html

In [ ]:
model_vect = DictVectorizer()
docs_vect = model_vect.fit_transform([Counter(doc['body_tokens']) for doc in documents])

In [ ]:
print "Documentos:{}\nPalabras:{}".format(*docs_vect.get_shape())

In [ ]:
model_vect.get_feature_names()

In [ ]:
assert(len(documents[0]['body_tokens'])==docs_vect[0].sum())

In [ ]:
print docs_vect[0].sum()

In [ ]:
print docs_vect[0].toarray()  # Sparse representation

## Similitud entre documentos

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def find_most_similars(doc_id, docs_vect, n=1):
    '''Return the most (n) similar documents from the list
    '''
    rank = sorted(enumerate(cosine_similarity(docs_vect[doc_id], docs_vect)[0]), reverse=True, key= lambda (_,similarity): similarity)
    rank = [x for x in rank if x[0]!=doc_id] # Filter original doc
    return rank[:n]

In [ ]:
find_most_similars(0, docs_vect, 4)

In [ ]:
all_sims = cosine_similarity(docs_vect, docs_vect)  # NxN document similarity matrix
np.fill_diagonal(all_sims,0)  # The most similar document is itself, set to 0
most_similar = all_sims.argmax(axis=1)
similarities = []  # (doc,doc, similarity) for most similar document to each one
for n_doc in xrange(len(documents)):
    sim_doc = most_similar[n_doc]
    similarities.append( (n_doc, sim_doc, all_sims[n_doc, sim_doc]) )
similarities.sort(key=lambda x:x[2], reverse=True)

In [ ]:
similarities[:20]

Parece que hay documentos muy parecidos.

In [ ]:
# Quitar duplicados, mismo texto y autor

similarities = [
    (doc1,doc2,sim)
    for doc1,doc2,sim in similarities
    if not (
        documents[doc1]['body'] == documents[doc2]['body'] and
        documents[doc1]['author'] == documents[doc2]['author']
    )
]

similarities[:20]

In [ ]:
sns.distplot([x[2] for x in similarities])

### Encontrar copiones de comentarios

In [ ]:
def find_copycats(similarities, threshold):
    """Documentos muy parecidos con diferentes autores"""
    retval = []
    for doc1,doc2,score in similarities:
        if score>threshold and documents[doc1]['author']!=documents[doc2]['author']:
            retval.append((doc1,doc2,score))
    return retval

In [ ]:
find_copycats(similarities,0.65)

In [ ]:
for doc1,doc2,score in find_copycats(similarities,0.65):
    print u"Authors: {}-{}, {}".format(documents[doc1]['author'], documents[doc2]['author'], score)

### Los dos documentos más parecidos

In [ ]:
def print_doc(doc):
    print doc['author']
    print doc['title']
    print doc['body']

doc1, doc2, score = [s for s in similarities if s[2]<0.65][0]
print "Score: {}\n".format(score)
print_doc(documents[doc1])
print ""
print_doc(documents[doc2])

### Ejecución de consultas

In [ ]:
# Función de pasar textos a tokens, del cuaderno anterior

STOPWORDS = set('''
a al algo algunas algunos ante antes como con contra cual cuando de del desde donde durante e el ella ellas ellos
en entre era erais eran eras eres es esa esas ese eso esos esta estaba estabais estaban estabas estad estada estadas
estado estados estamos estando estar estaremos estara estaran estaras estare estareis estaria estariais estariamos
estarian estarias estas este estemos esto estos estoy estuve estuviera estuvierais estuvieran estuvieras estuvieron
estuviese estuvieseis estuviesen estuvieses estuvimos estuviste estuvisteis estuvieramos estuviesemos estuvo esta
estabamos estais estan estas este esteis esten estes fue fuera fuerais fueran fueras fueron fuese fueseis fuesen fueses
fui fuimos fuiste fuisteis fueramos fuesemos ha habida habidas habido habidos habiendo habremos habra habran habras
habre habreis habria habriais habriamos habrian habrias habeis habia habiais habiamos habian habias han has hasta
hay haya hayamos hayan hayas hayais he hemos hube hubiera hubierais hubieran hubieras hubieron hubiese hubieseis
hubiesen hubieses hubimos hubiste hubisteis hubieramos hubiesemos hubo la las le les lo los me mi mis mucho muchos
muy mas mi mia mias mio mios nada ni no nos nosotras nosotros nuestra nuestras nuestro nuestros o os otra otras otro
otros para pero poco por porque que quien quienes que se sea seamos sean seas sentid sentida sentidas sentido sentidos
seremos sera seran seras sere sereis seria seriais seriamos serian serias seais siente sin sintiendo sobre sois somos
son soy su sus suya suyas suyo suyos si tambien tanto te tendremos tendra tendran tendras tendre tendreis tendria
tendriais tendriamos tendrian tendrias tened tenemos tenga tengamos tengan tengas tengo tengais tenida tenidas tenido
tenidos teniendo teneis tenia teniais teniamos tenian tenias ti tiene tienen tienes todo todos tu tus tuve tuviera
tuvierais tuvieran tuvieras tuvieron tuviese tuvieseis tuviesen tuvieses tuvimos tuviste tuvisteis tuvieramos tuviesemos
tuvo tuya tuyas tuyo tuyos tu un una uno unos vosostras vosostros vuestra vuestras vuestro vuestros y ya yo el eramos
'''.split())

from nltk.stem.snowball import SpanishStemmer
stemmer = SpanishStemmer()

def txt2words(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [
        stemmer.stem(w)
        for w in txt.split(' ')
        if w!='' and w not in STOPWORDS
    ]
    return words

In [ ]:
def query_vect(text):
    doc = model_vect.transform(Counter(txt2words(text)))
    similarities = cosine_similarity(doc, docs_vect)
    doc_id,score = similarities.argmax(), similarities.max()    
    print doc_id
    print u"Pelicula: '{}'\nPuntuacion:{}".format(documents[doc_id]['title'],score)

In [ ]:
# Tematica correcta
query_vect(u'quiero una pelicula de boxeo')
query_vect(u'quiero una peli de boxeo')

In [ ]:
# Understemming
query_vect(u'quiero una pelicula triste')
query_vect(u'quiero una peli triste')

In [ ]:
query_vect(u'Algo de rambo')

In [ ]:
query_vect(u'ciencia ficción futurista')

In [ ]:
query_vect(u'amor romántico bobo')

In [ ]:
query_vect(u'americanada universidad')

## TF-IDF (Term Frequency - Inverse Document Frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
model_tfidf = TfidfTransformer()
docs_tfidf = model_tfidf.fit_transform(docs_vect)

def query_tfidf(text):
    doc,score = query_documents(text, model_tfidf, docs_tfidf)
    print doc
    print u"Pelicula: '{}'\nPuntuacion:{}".format(documents[doc]['title'],score)

In [ ]:
def query_tfidf(text):
    doc = model_tfidf.transform(  # IDF learn
            model_vect.transform(  # Matrix representation
                Counter(txt2words(text))  # Dict of frequencies
            )
    )
    similarities = cosine_similarity(doc, docs_tfidf)
    doc_id,score = similarities.argmax(), similarities.max()    
    print doc_id
    print u"Pelicula: '{}'\nPuntuacion:{}".format(documents[doc_id]['title'],score)

In [ ]:
# Normalizacion peli -pelicula
query_tfidf(u'quiero una pelicula de boxeo')
query_tfidf(u'quiero una peli de boxeo')

In [ ]:
# Understemming
query_tfidf(u'quiero una pelicula triste')
query_tfidf(u'quiero una peli triste')

In [ ]:
query_tfidf(u'')

In [ ]:
query_tfidf(u'ciencia ficción futurista')